### 練習問題2.57
微分プログラムを拡張し、(2以上の)任意の数の項の和と積を扱えるようにせよ。  
すると上の最後の例は以下のように表現できる。

    (deriv '(* x y (+ x 3)) 'x)

deriv⼿続きにはまったく⼿を加えず、
和と積の表現のみを変更することによって解け。  
例えば、和のaddend(加数)は最初の項で、  
augend(被加数)は残りの項の和というようにする。

In [1]:
(define (=number? exp num)(and (number? exp) (= exp num)))

; eは変数か?
(define (variable? x)(symbol? x)) 

; v1とv2は同じ変数か?
(define (same-variable? v1 v2)(and (variable? v1) (variable? v2) (eq? v1 v2)))

; eは和か?
(define (sum? x)(and (pair? x) (eq? (car x) '+)))

; 和eの加数
(define (addend s)(cadr s))

; 和eの被加数
;(define (augend s)(caddr s))
(define (augend p)
  (define (iter t1 t2)
    (cond ((and (null? t1) (null? t2)) '())
          ((and (not (null? t1)) (null? t2)) t1)
          (else (make-sum t1 (iter (car t2) (cdr t2))))
          )
    )
  (iter (caddr p) (cdddr p))
  )

; a1とa2の和を構築する
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2)))
  )

;eは積か?
(define (product? x)(and (pair? x) (eq? (car x) '*))) 

; 積eの乗数
(define (multiplier p)(cadr p))

;積eの被乗数
;(define (multiplicand p)(caddr p))
(define (multiplicand p)
  (define (iter t1 t2)
    (cond ((and (null? t1) (null? t2)) '())
          ((and (not (null? t1)) (null? t2)) t1)
          (else (make-product t1 (iter (car t2) (cdr t2))))
          )
    )
  (iter (caddr p) (cdddr p))
  )

; m1とm2の積を構築する
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))
    )
  )

;eはべき乗か?
(define (exponentiation? x)
  (and (pair? x) (eq? (car x) '**))) 

; べき乗の基数
(define (base x)(cadr x))

; 指数
(define (exponent x)(caddr x))

; べき数を構築する
(define (make-exponentiation m1 m2)
  (cond ((=number? m2 0) 1)
        ((=number? m2 1) m1)
        ;((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2) (> m2 0)) ((make-exponentiation (* m1 m1) (- m2 1))))
        (else (list '** m1 m2)))
  )


(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum (make-product (multiplier exp)
                                                (deriv (multiplicand exp) var))
                                  (make-product (deriv (multiplier exp) var) (multiplicand exp))))
        ((exponentiation? exp)
         (let ((u (base exp))
               (e (exponent exp)))
            (make-product (make-product e (make-exponentiation u (- e 1))) (deriv u var))
           )
         )
        (else (error "unknown expression type: DERIV" exp)))
  )

In [2]:
(deriv '(* x x) 'x)

(+ x x)

In [3]:
(deriv '(* x x x) 'x)

(+ (* x (+ x x)) (* x x))

In [4]:
(deriv '(* x x x x) 'x)

(+ (* x (+ (* x (+ x x)) (* x x))) (* x (* x x)))

In [5]:
(deriv '(* x x x x x) 'x)

(+ (* x (+ (* x (+ (* x (+ x x)) (* x x))) (* x (* x x)))) (* x (* x (* x x))))

In [6]:
(deriv '(+ x 3) 'x)

1

In [7]:
(deriv '(+ x x x x x) 'x)

5

In [8]:
(deriv '(* x y) 'x)

y

In [9]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

In [10]:
(deriv '(** x 3) 'x)

(* 3 (** x 2))

In [11]:
(deriv '(** x 2) 'x)

(* 2 x)

In [12]:
(deriv '(** x 1) 'x)

1

In [13]:
(deriv '(** (+ x 1) 2) 'x)

(* 2 (+ x 1))

In [14]:
(deriv '(* x y (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

In [15]:
(length (list 1 2))

2

In [16]:
(deriv '(* x x) 'x)

(+ x x)

In [17]:
(deriv '(* x x x) 'x)

(+ (* x (+ x x)) (* x x))

In [18]:
(deriv '(* x x x x) 'x)

(+ (* x (+ (* x (+ x x)) (* x x))) (* x (* x x)))